In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    status_file: Path
    ALL_REQUIRED_FILES: list[str]

: 

In [ ]:
from src.constants import *
from src.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([Path(self.config.artifacts_root)])

    

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_ingestion

        create_directories([Path(config.root_dir)])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            status_file=config.STATUS_FILE
            ALL_REQUIRED_FILES: config.ALL_REQUIRED_FILES
        )

        return data_validation_config

In [ ]:
import os
from src.logging import logger

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    ## validate all files are present, and add the value to the status file

    def validate_all_files(self) -> bool:
        try:
            logger.info("Starting data validation")
            all_files_present = True
            for file_name in self.config.ALL_REQUIRED_FILES:
                file_path = os.path.join(self.config.root_dir, file_name)
                if not os.path.exists(file_path):
                    logger.error(f"Required file {file_name} is missing.")
                    all_files_present = False
                else:
                    logger.info(f"Required file {file_name} is present.")
            
            with open(self.config.status_file, 'w') as status_file:
                if all_files_present:
                    status_file.write("Data Validation Successful: All required files are present.\n")
                    logger.info("Data validation successful: All required files are present.")
                else:
                    status_file.write("Data Validation Failed: Some required files are missing.\n")
                    logger.error("Data validation failed: Some required files are missing.")
            
            return all_files_present

        except Exception as e:
            logger.exception(f"An error occurred during data validation: {e}")
            raise e

In [ ]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files()    
except Exception as e:
    raise e